In [1]:
import pandas as pd
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Activation,Dropout
from keras.layers import Embedding
from keras.layers import SimpleRNN, TimeDistributed
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.callbacks import LearningRateScheduler
from keras.callbacks import History

from keras import losses
from sklearn.utils import shuffle

Using TensorFlow backend.


In [2]:
csv_data_df=pd.read_csv("query-data/br-query-corpus.csv", encoding='latin-1')
# csv_data_df = shuffle(csv_data_df)
features = csv_data_df.iloc[:, 0]
print(features.head())
classes = csv_data_df.iloc[:, 1]
print(classes.head())

0    Bug bot improve documentation feedback error h...
1    Bug Remotesvcs OSGi ServiceUIComponent violate...
2    Bug org eclipse ecf internal discovery Discove...
3         Bug Discovery JmDNS Timer canceled early DNS
4                        Bug IRC tab completion broken
Name: query, dtype: object
0    good
1    good
2    good
3    good
4    good
Name: classlabel, dtype: object


In [3]:
#tokenize the texts to number
from keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer()
tokenizer.fit_on_texts(features)
X = tokenizer.texts_to_sequences(features)

In [4]:
def get_encoded_labels(yvalues):
    values = array(yvalues)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    onehot_encoder = OneHotEncoder(sparse=False, categories="auto")
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded_labels = onehot_encoder.fit_transform(integer_encoded)
    return onehot_encoded_labels

In [5]:
Y=get_encoded_labels(classes)

In [6]:
#showing the tokenized features and classes
print(X[0])
print(Y[0])

print(len(tokenizer.index_word))



[1, 878, 414, 879, 1584, 6, 227]
[0. 1.]
3498


In [7]:
print('Maximum review length: {}'.format(
len(max((X), key=len))))

Maximum review length: 25


In [8]:
#padding the queries 
max_words=25

#padding the documents
from keras.preprocessing import sequence
padded_X = sequence.pad_sequences(X, maxlen=max_words)

print(padded_X[1])


[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    1 1119  121 1585  880 1120   67 1586]


In [9]:
#developing training and test set
from sklearn.model_selection import train_test_split
import numpy as np

x_train,x_test,y_train,y_test = train_test_split(padded_X, Y, test_size=.20, random_state=100)
x_train.shape[1], y_test.shape[1]

(25, 2)

In [10]:
#constructing the LSTM model
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

embedding_size=100
vocabulary_size=3500

model=Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(2, activation="relu"))
model.add(Dense(2, activation='sigmoid'))

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 100)           350000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 6         
Total params: 430,608
Trainable params: 430,608
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
# compile the model
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [15]:
#model training
batch_size = 32
num_epochs = 25

x_valid, y_valid = x_train[:batch_size], y_train[:batch_size]
x_train2, y_train2 = x_train[batch_size:], y_train[batch_size:]

In [16]:
x_valid[0]
# y_valid[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   39,
       3128,   58, 3129])

In [17]:
#fitting the model
model.fit(x_train2, y_train2, validation_data=(x_valid, y_valid), 
          batch_size=batch_size, epochs=num_epochs)

c:\users\masudrahman\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1824 samples, validate on 32 samples
Epoch 1/25
1824/1824 [==============================] - 2s 955us/step - loss: 0.6904 - accuracy: 0.5872 - val_loss: 0.7002 - val_accuracy: 0.4219
Epoch 2/25
1824/1824 [==============================] - 1s 694us/step - loss: 0.6401 - accuracy: 0.6859 - val_loss: 0.8472 - val_accuracy: 0.4531
Epoch 3/25
1824/1824 [==============================] - 1s 769us/step - loss: 0.4909 - accuracy: 0.7867 - val_loss: 0.7986 - val_accuracy: 0.5625
Epoch 4/25
1824/1824 [==============================] - 1s 764us/step - loss: 0.3168 - accuracy: 0.8857 - val_loss: 1.0589 - val_accuracy: 0.6094
Epoch 5/25
1824/1824 [==============================] - 1s 723us/step - loss: 0.1847 - accuracy: 0.9378 - val_loss: 1.2604 - val_accuracy: 0.6094
Epoch 6/25
1824/1824 [==============================] - 2s 917us/step - loss: 0.1009 - accuracy: 0.9679 - val_loss: 1.4379 - val_accuracy: 0.5625
Epoch 7/25
1824/1824 [==============================] - 2s 1ms/step - loss: 0.